In [24]:
import numpy as np
import numba
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## Pure python + numpy

In [40]:
def primes(kmax):
    result = []
    p = np.zeros((kmax))
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result


In [41]:
%timeit primes(1000)

1 loop, best of 3: 260 ms per loop


## cython

In [42]:
%%cython

def primes_cython(int kmax):
    cdef int n, k, i
    cdef int p[1000]
    result = []
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [43]:
%timeit primes_cython(1000)

100 loops, best of 3: 2.14 ms per loop


In [44]:
3.06/0.027

113.33333333333334

## numba + numpy

In [48]:
@numba.jit(nopython=True)
def primes_numba(kmax):
    result = []
    p = np.zeros((kmax))
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [46]:
%timeit primes_numba(1000)

The slowest run took 28.21 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 5.71 ms per loop


In [88]:
%%cython
import numpy as np
cimport numpy as np
def suma_cython(np.ndarray a, np.ndarray b):   
    return np.sum(np.sum(a+b))

In [89]:
a = np.linspace(0,10,1000)
b = np.linspace(0,10,1000)
%timeit suma_cython(a,b)

The slowest run took 5.75 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 18.4 µs per loop


In [90]:
def suma(a,b):
    return np.sum(np.sum(a+b))

In [91]:
%timeit suma(a,b)

The slowest run took 4.31 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 18.2 µs per loop


In [92]:
@numba.jit(nopython=True)
def suma_numba(a,b): 
    return np.sum(np.sum(a+b))

In [93]:
a = np.linspace(0,10,1000)
b = np.linspace(0,10,1000)
%timeit suma_numba(a,b)

TypingError: Failed at nopython (nopython frontend)
Invalid usage of Function(<function sum at 0x7f1fb8077620>) with parameters (float64)
 * parameterized
File "<ipython-input-92-359cf2fd04b1>", line 3
[1] During: resolving callee type: Function(<function sum at 0x7f1fb8077620>)
[2] During: typing of call at <ipython-input-92-359cf2fd04b1> (3)

In [166]:
%%cython
import numpy as np
cimport numpy as np
class clase_cython(object):
    def __init__(self, N):
        self.N = N
        self.A = np.ones((N,N))
        self.B = np.ones((N,N))*3

cdef suma_cy(clase):
    cdef double s = 0.0
    cdef int ir
    cdef int ic
    
    cdef np.ndarray C = clase.A @ clase.B
    for ir in range(clase.N):
        for ic in range(clase.N):
            s += np.sin(C[ir,ic]) 

    return s

In [167]:
obj_cython = clase_cython(1000)

In [199]:
%%cython
import numpy as np
cimport numpy as np

cdef np.ndarray[np.double_t, ndim=2] suma_cy_simple(np.ndarray[np.double_t, ndim=2] A, np.ndarray[np.double_t, ndim=2] B, int N):
    cdef double s = 0.0
    cdef int ir
    cdef int ic 
    cdef np.ndarray C = A @ B
    for ir in range(N):
        for ic in range(N):
            s += np.sin(A[ir,ic]) 

    return s

In [200]:
N = 100
%timeit suma_cy_simple(np.ones((N,N)),np.ones((N,N))*3,N)

100 loops, best of 3: 13.4 ms per loop


In [150]:
class clase(object):
    def __init__(self, N):
        self.N = N
        self.A = np.ones((N,N))
        self.B = np.ones((N,N))*3

def suma(clase):
    s = 0.0
    C = clase.A @ clase.B
    for ir in range(clase.N):
        for ic in range(clase.N):
            s += np.sin(C[ir,ic]) 
            
    return s

In [151]:
obj = clase(1000)

In [152]:
%timeit suma(obj)

1 loop, best of 3: 1.49 s per loop


In [132]:
%%cython
cdef class Function:
    cpdef double evaluate(self, double x) except *:
        return 2*x


In [133]:
obj = Function(10.0)

In [134]:
obj.evaluate(2)

4.0

In [149]:
%%cython
import numpy as np
cimport numpy as np

cdef class Function:
    cpdef np.ndarray evaluate(self, np.ndarray x) except *:
        return 0

cdef class SinOfSquareFunction(Function):
    cpdef np.ndarray evaluate(self, np.ndarray x) except *:
        return np.sin(x**2)

def integrate(Function f, np.ndarray x):
    def suma(self):
        s = 0.0
        for ir in range(10):
            for ic in range(10):
                s += np.sin(x[ir,ic]) 

print(integrate(SinOfSquareFunction(), 0, 1, 10000))



Error compiling Cython file:
------------------------------------------------------------
...
import numpy as np
cimport numpy as np

cdef class Function:
    cpdef np.ndarray evaluate(self, np.ndarray x) except *:
                            ^
------------------------------------------------------------

/home/jmmauricio/.cache/ipython/cython/_cython_magic_c238f11ca3e408b580349cfc3e410a74.pyx:5:29: Exception clause not allowed for function returning Python object

Error compiling Cython file:
------------------------------------------------------------
...
cdef class Function:
    cpdef np.ndarray evaluate(self, np.ndarray x) except *:
        return 0

cdef class SinOfSquareFunction(Function):
    cpdef np.ndarray evaluate(self, np.ndarray x) except *:
                            ^
------------------------------------------------------------

/home/jmmauricio/.cache/ipython/cython/_cython_magic_c238f11ca3e408b580349cfc3e410a74.pyx:9:29: Exception clause not allowed for function retu

In [196]:
def suma_py_simple(A,B,N):
    C = A @ B
    s=0.0
    for ir in range(N):
        for ic in range(N):
            s += np.sin(A[ir,ic]) 

    return s


In [198]:
N = 100
%timeit suma_py_simple(np.ones((N,N)),np.ones((N,N))*3,N)

100 loops, best of 3: 14.8 ms per loop


In [202]:
import numpy as np
def naive_convolve(f, g):
    # f is an image and is indexed by (v, w)
    # g is a filter kernel and is indexed by (s, t),
    #   it needs odd dimensions
    # h is the output image and is indexed by (x, y),
    #   it is not cropped
    if g.shape[0] % 2 != 1 or g.shape[1] % 2 != 1:
        raise ValueError("Only odd dimensions on filter supported")
    # smid and tmid are number of pixels between the center pixel
    # and the edge, ie for a 5x5 filter they will be 2.
    #
    # The output size is calculated by adding smid, tmid to each
    # side of the dimensions of the input image.
    vmax = f.shape[0]
    wmax = f.shape[1]
    smax = g.shape[0]
    tmax = g.shape[1]
    smid = smax // 2
    tmid = tmax // 2
    xmax = vmax + 2*smid
    ymax = wmax + 2*tmid
    # Allocate result image.
    h = np.zeros([xmax, ymax], dtype=f.dtype)
    # Do convolution
    for x in range(xmax):
        for y in range(ymax):
            # Calculate pixel value for h at (x,y). Sum one component
            # for each pixel (s, t) of the filter g.
            s_from = max(smid - x, -smid)
            s_to = min((xmax - x) - smid, smid + 1)
            t_from = max(tmid - y, -tmid)
            t_to = min((ymax - y) - tmid, tmid + 1)
            value = 0
            for s in range(s_from, s_to):
                for t in range(t_from, t_to):
                    v = x - smid + s
                    w = y - tmid + t
                    value += g[smid - s, tmid - t] * f[v, w]
            h[x, y] = value
    return h

In [204]:
N = 100
f = np.arange(N*N, dtype=np.int).reshape((N,N))
g = np.arange(81, dtype=np.int).reshape((9, 9))
%timeit naive_convolve(f, g)

1 loop, best of 3: 561 ms per loop


In [210]:
%%cython
import numpy as np

def cy_convolve(f, g):
    # f is an image and is indexed by (v, w)
    # g is a filter kernel and is indexed by (s, t),
    #   it needs odd dimensions
    # h is the output image and is indexed by (x, y),
    #   it is not cropped
    if g.shape[0] % 2 != 1 or g.shape[1] % 2 != 1:
        raise ValueError("Only odd dimensions on filter supported")
    # smid and tmid are number of pixels between the center pixel
    # and the edge, ie for a 5x5 filter they will be 2.
    #
    # The output size is calculated by adding smid, tmid to each
    # side of the dimensions of the input image.
    vmax = f.shape[0]
    wmax = f.shape[1]
    smax = g.shape[0]
    tmax = g.shape[1]
    smid = smax // 2
    tmid = tmax // 2
    xmax = vmax + 2*smid
    ymax = wmax + 2*tmid
    # Allocate result image.
    h = np.zeros([xmax, ymax], dtype=f.dtype)
    # Do convolution
    for x in range(xmax):
        for y in range(ymax):
            # Calculate pixel value for h at (x,y). Sum one component
            # for each pixel (s, t) of the filter g.
            s_from = max(smid - x, -smid)
            s_to = min((xmax - x) - smid, smid + 1)
            t_from = max(tmid - y, -tmid)
            t_to = min((ymax - y) - tmid, tmid + 1)
            value = 0
            for s in range(s_from, s_to):
                for t in range(t_from, t_to):
                    v = x - smid + s
                    w = y - tmid + t
                    value += g[smid - s, tmid - t] * f[v, w]
            h[x, y] = value
    return h

In [212]:
%timeit cy_convolve(f, g)

1 loop, best of 3: 411 ms per loop


In [215]:


from numba.pycc import CC

cc = CC('my_module')
# Uncomment the following line to print out the compilation steps
cc.verbose = True

@cc.export('multf', 'f8(f8, f8)')
@cc.export('multi', 'i4(i4, i4)')
def mult(a, b):
    return a * b

@cc.export('square', 'f8(f8)')
def square(a):
    return a ** 2